In [ ]:
2.1.1 Eager Execution
 Eager (이거)는 기존의 세션과 그래프 형식의 작동 방식에서 명령형 (imperative) 스타일로 텐서플로우를 활용할수 있는 기능이다.  기존에 텐서플로우 방식이 세션과 그래프 완성 후 디버깅을 해야하는 불편함이 있었다면, 이거 모드를 통해서 좀 더 직관적으로 접근 가능하다.  따라서 연구자들이 조금 더 유동적으로 텐서플로우 프레임워크를 활용하거나 새롭게 텐서플로우를 입문하시는 분들에게 적합한 기능이다.
텐서플로우 2018 Dev Summit에서 내용을 발췌하면 다음과 같다.
 - Eager 모드는 점점 contrib에서 정식으로 변하고 있고 session을 사용하지 않고 코드를 실행 가능하다.
 - 그래디언트 계산을 손쉽게 수정이 가능하다.
 - Dataset (텐서플로우 자료 구조 중 하나)를 통해서 sqlite database를 읽을 수 있다 (아직 실험적)
 - TensorRT를 지원함으로써 모델을 최적화 할 수 있다.
이 장에서는 Eager 모드에 대한 간략한 소개, 기능 설명 및 예제를 제공하겠다. 우선 eager 모드를 실행하는 방법은 아래와 같다.
import tensorflow as tf

# eager 모드를 실행하는 방법
# eager 모드는 프로그램 시작 때 실행을 해야하며, 재시작 까지는 eager 모드가 유지됩니다.
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))


출력 결과:
TensorFlow version: 1.8.0
Eager execution: True


In [ ]:
직관적인 연산
  Eager 모드를 통해, 기존의 세션 방식과 비교하여 조금 더 직관적으로 연산이 가능하다. 2 * 2 매트릭스에서 연산을 하는 예를 들어 보자.
x = tf.matmul([[1, 2],
               [3, 4]],
              [[4, 5],
               [6, 7]])

y = tf.add(x, 1)

print(x)
### 매트릭스 연산 ###
tf.Tensor(
[[16 19]
 [36 43]], shape=(2, 2), dtype=int32)
print(y)
### 매트릭스 덧샘 ###
tf.Tensor(
[[17 20]
 [37 44]], shape=(2, 2), dtype=int32)

### 텐서플로우 자료형에서 넘파이 자료형태로 변환
x.numpy()

array([[16, 19],
       [36, 43]], dtype=int32)
 
x.numpy() #넘파이 형태로 변환도 가능합니다.


#반대로 tf.constant 활용하여 넘파이에서 텐서플로우 구조로도 변화가 가능합니다.

np_val = np.array(10., dtype=np.float32)
tf_val = tf.constant(np_val) #tf.constant를 사용하여 넘파이 데이터 변환

print(np_val)
print(tf_val)
10.0
tf.Tensor(10.0, shape=(), dtype=float32)

## Define and Print Tensorflow Variables

x = tf.get_variable(name="x", shape=[], dtype=tf.float32, initializer=tf.zeros_initializer)
print(x)

#Tensorflow의 변수는 tensor로 나타냄으로, read_value()를 통해 현재 값으로 접근이 가능함.
#Tensorflow의 함수는 자동으로 초기화
<tf.Variable 'x:0' shape=() dtype=float32, numpy=0.0>
In [33]:
#numpy를 통한 변환
print(x.read_value().numpy())

#Tensorflow변수의 값을 변경하기
x.assign(42)
print(x.assign)

x.assign_add(3) #x 값에 더하기 적용
print(x.read_value())

print(x + 3) #텐서 변수를 자유자제로 활용해보기

print(x * [1, 2, 4]) #자동으로 broadcasting도 가능함
45.0
<bound method ResourceVariable.assign of <tf.Variable 'x:0' shape=() dtype=float32, numpy=42.0>>
tf.Tensor(45.0, shape=(), dtype=float32)
tf.Tensor(48.0, shape=(), dtype=float32)
tf.Tensor([ 45.  90. 180.], shape=(3,), dtype=float32)
## Automatic Difference (Gradients)

 - tfe.gradients_function(f): 입력 f에 대해 arg 미분값을 돌려준다.
 - tfe.value_and_gradients_function(f): tfe.gradients_function(f)과 비슷하지만, 함수가 들어오면 이전 f값과 미분값에 대해 값을 출력한다.
def square(x):
    return tf.multiply(x, x)
assert 9 == square(3.).numpy()

grad = tfe.gradients_function(square)
assert 6 == grad(3.)[0].numpy()

print(square(3.))
print(grad(3.)) #x^2 -> 2x -> 6
tf.Tensor(9.0, shape=(), dtype=float32)
[<tf.Tensor: id=14844, shape=(), dtype=float32, numpy=6.0>]
#2차 gradients_function
grad2 = tfe.value_and_gradients_function(lambda x: grad(x)[0])
#assert 2 == grad2(3.)[0].numpy()
print("2nd grad: {}".format(grad2(3.)))

#3차 grad.
grad3 = tfe.gradients_function(lambda x: grad2(x)[0])
#assert 0 == grad3(3.)[0].numpy()
print(grad3(3.))

#absolute value
def abs(x):
    return x if x > 0. else -x

grad = tfe.gradients_function(abs)

print(grad(2.0))  # [1.]
print(grad(-2.0)) # [-1.]
2nd grad: (<tf.Tensor: id=14851, shape=(), dtype=float32, numpy=6.0>, [<tf.Tensor: id=14856, shape=(), dtype=float32, numpy=2.0>])
[<tf.Tensor: id=14875, shape=(), dtype=float32, numpy=2.0>]
[<tf.Tensor: id=72, shape=(), dtype=float32, numpy=1.0>]
[<tf.Tensor: id=14887, shape=(), dtype=float32, numpy=-1.0>]

#실제 linear regression을 통하여 활용해보자

def prediction(input, weight, bias):
    return input * weight + bias

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# A loss function: Mean-squared error
def loss(weight, bias):
    error = prediction(training_inputs, weight, bias) - training_outputs
    return tf.reduce_mean(tf.square(error))

# Function that returns the the derivative of loss with respect to
# weight and bias
grad = tfe.gradients_function(loss)

# Train for 200 steps (starting from some random choice for W and B, on the same
# batch of data).
W = 5.
B = 10.
learning_rate = 0.01
print("Initial loss: %f" % loss(W, B).numpy())
for i in range(200):
    (dW, dB) = grad(W, B)
    W -= dW * learning_rate
    B -= dB * learning_rate
    if i % 20 == 0:
        print("Loss at step %d: %f" % (i, loss(W, B).numpy()))
print("Final loss: %f" % loss(W, B).numpy())
print("W, B = %f, %f" % (W.numpy(), B.numpy()))
Initial loss: 69.151985
Loss at step 0: 66.454216
Loss at step 20: 30.185934
Loss at step 40: 14.019278
Loss at step 60: 6.812951
Loss at step 80: 3.600699
Loss at step 100: 2.168821
Loss at step 120: 1.530553
Loss at step 140: 1.246040
Loss at step 160: 1.119215
Loss at step 180: 1.062682
Final loss: 1.038317
W, B = 3.034794, 2.132014

## Building and training models

 - eager에서는 특별히 수정해야 하지 않는 한, tf.layers와 같은 모듈을 사용을 권장함
 - Optimizer와 layer를 간단하게 정리
## Variable & Optimization

 - tfe.Variable: 변형가능한 Tensor값을 저장하는 객체로써, 학습이나 미분을 할때 값에 대한 access가 가능함. 모델의 파라메터들이 python변수에 저장 될 수 있다는 이야기임
 - tfe.gradients_function(f): 쉬운 미분을 지원하지만, 모든 파라메터들이 f와 연동이 되어있어야 하여, 학습시 큰 파라메터에 대한 대응이 힘듬
 - tfe.implicit_gradients: 비슷한 기능이지만 몇가지 특수 기능이 있음?

#실제 linear regression을 통하여 활용해보자

class Model(object):
    def __init__(self):
        self.W = tfe.Variable(5., name='weight')
        self.B = tfe.Variable(10., name='bias')

    def predict(self, inputs):
        return inputs * self.W + self.B


# The loss function to be optimized
def loss(model, inputs, targets):
    error = model.predict(inputs) - targets
    return tf.reduce_mean(tf.square(error))

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# Define:
# 1. A model
# 2. Derivatives of a loss function with respect to model parameters
# 3. A strategy for updating the variables based on the derivatives
model = Model()
grad = tfe.implicit_gradients(loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

# The training loop
print("Initial loss: %f" %
      loss(model, training_inputs, training_outputs).numpy())
for i in range(201):
    optimizer.apply_gradients(grad(model, training_inputs, training_outputs))
    if i % 20 == 0:
        print("Loss at step %d: %f" %
              (i, loss(model, training_inputs, training_outputs).numpy()))
print("Final loss: %f" % loss(model, training_inputs, training_outputs).numpy())
print("W, B = %s, %s" % (model.W.numpy(), model.B.numpy()))

Initial loss: 67.940033
Loss at step 0: 65.334366
Loss at step 20: 30.092083
Loss at step 40: 14.164181
Loss at step 60: 6.957934
Loss at step 80: 3.694184
Loss at step 100: 2.214460
Loss at step 120: 1.542879
Loss at step 140: 1.237761
Loss at step 160: 1.098992
Loss at step 180: 1.035816
Loss at step 200: 1.007025
Final loss: 1.007025
W, B = 3.0174496, 2.1403313
 
Eager Gradients
대부분의 TensorFlow 사용자는 자동 차별화에 관심이 있습니다.
각 호출 중에 다른 작업이 발생할 수 있으므로 모든 포워드 작업을 테이프에 기록한 다음 그래디언트를 계산할 때 역방향으로 재생합니다.
그라디언트를 계산 한 후에 테이프를 버립니다.
autograd 패키지에 익숙하다면 API가 매우 유사합니다. 
def square(x):
    return tf.multiply(x, x)
​grad = tfe.gradients_function(square)
​print(square(3.))    # [9.]
print(grad(3.))      # [6.]        
 
gradients_function 은 인자로 파이썬 함수 square()를 사용하고 그 입력에 대해 square()의 편도를 계산하는 파이썬 호출 가능 객체를 반환합니다. 따라서 square ()의 미분을 3.0으로 얻으려면 grad (3.0)을 호출합니다.
이것은 6입니다.
동일한 gradients_function 호출을 사용하여 square의 2 차 미분을 얻을 수 있습니다.
gradgrad = tfe.gradients_function(lambda x: grad(x)[0])
​print(gradgrad(3.))  # [2.]
​def abs(x):
     return x if x > 0. else -x
​grad = tfe.gradients_function(abs)
​print(grad(2.0))  # [1.]
print(grad(-2.0)) # [-1.]
​x = tfe.Variable(2.0)
def loss(y):
    return (y - x ** 2) ** 2
​grad = tfe.implicit_gradients(loss)
​print(loss(7.))  # tf.Tensor(9., shape=(), dtype=float32)
print(grad(7.))  # [(<tf.Tensor: -24.0, shape=(), dtype=float32>,
                    <tf.Variable 'Variable:0' shape=()               
                     dtype=float32, numpy=2.0>)]
 
why grad(7.)이 -24가 나오는지 아시나요?  (edit)
eager 실행이 활성화되지 않은 경우에도 그라디언트  API가 작동합니다.
tfe.gradients_function()
tfe.value_and_gradients_function()
tfe.implicit_gradients()
tfe.implicit_value_and_gradients()
​
이 부분에 대해서 좀 더 보충 설명 하겠습니다.  (edit)
